# Módulo 5 Tarefa 1
## Base de nascidos vivos do DataSUS
O DataSUS disponibiliza diversos arquivos de dados com relação a seus segurados, conforme a [lei da transparência de informações públicas](https://www.sisgov.com/transparencia-acesso-informacao/#:~:text=A%20Lei%20da%20Transpar%C3%AAncia%20(LC,em%20um%20site%20na%20internet.). 

Essas informações podem ser obtidas pela internet [aqui](http://www2.datasus.gov.br/DATASUS/index.php?area=0901&item=1). Como o processo de obtenção desses arquivos foge um pouco do nosso escopo, deixamos o arquivo ```SINASC_RO_2019.csv``` já como vai ser encontrado no DataSUS. O dicionário de dados está no arquivo ```estrutura_sinasc_para_CD.pdf``` (o nome do arquivo tal qual no portal do DataSUS).

### Nosso objetivo
Queremos deixar uma base organizada para podermos estudar a relação entre partos com risco para o bebê e algumas condições como tempo de parto, consultas de pré-natal etc.

#### Preparação da base
1. Carregue a base 'SINASC_RO_2019.csv'. Conte o número de registros e o número de registros não duplicados da base. Dica: você aprendeu um método que remove duplicados, encadeie este método com um outro método que conta o número de linhas. **Há linhas duplicadas?**  

2. Conte o número de valores *missing* por variável.  

3. Ok, no item anterior você deve ter achado pouco prático ler a informação de tantas variáveis, muitas delas nem devem ser interesantes. Então crie uma seleção dessa base somente com as colunas que interessam. São elas:
``` 
['LOCNASC', 'IDADEMAE', 'ESTCIVMAE', 'ESCMAE', 'QTDFILVIVO', 
    'GESTACAO', 'GRAVIDEZ', 'CONSULTAS', 'APGAR5'] 
```
Refaça a contagem de valores *missings*.  

4. Apgar é uma *nota* que o pediatra dá ao bebê quando nasce de acordo com algumas características associadas principalmente à respiração. Apgar 1 e Apgar 5 são as notas 1 e 5 minutos do nascimento. Apgar5 será a nossa variável de interesse principal. Então remova todos os registros com Apgar5 não preenchido. Para esta seleção, conte novamente o número de linhas e o número de *missings*.  

5. observe que as variáveis ```['ESTCIVMAE', 'CONSULTAS']``` possuem o código ```9```, que significa *ignorado*. Vamos assumir que o não preenchido é o mesmo que o código ```9```.<br>
6. Substitua os valores faltantes da quantitativa (```QTDFILVIVO```) por zero.  
7. Das restantes, decida que valore te parece mais adequado (um 'não preenchido' ou um valor 'mais provável' como no item anterior) e preencha. Justifique. Lembre-se de que tratamento de dados é trabalho do cientista, e que estamos tomando decisões a todo o momento - não há necessariamente certo e errado aqui.  
8. O Apgar possui uma classificação indicando se o bebê passou por asfixia:
- Entre 8 e 10 está em uma faixa 'normal'. 
- Entre 6 e 7, significa que o recém-nascido passou por 'asfixia leve'. 
- Entre 4 e 5 significa 'asfixia moderada'.
- Entre 0 e 3 significa 'asfixia severa'.  

Crie uma categorização dessa variável com essa codificação e calcule as frequências dessa categorização.  
<br>
9. Renomeie as variáveis para que fiquem no *snake case*, ou seja, em letras minúsculas, com um *underscore* entre as palávras. Dica: repare que se você não quiser criar um *dataframe* novo, você vai precisar usar a opção ```inplace = True```.

# 0. Indice <a name = "#Contents"></a>

0. [Importando os pacotes](#pacotes)
1. [Carregar a base](#base)
2. [Contar vazios por variavel](#2)
3. [Nova base e Recontagem](#3)
4. [Remover Apgar5 não preenchido](#4)
5. [Estado Civil e Consulta](#5)
6. [Substituir os valores faltantes (QTDFILVIVO)](#6)
7. [Substituir os valores das demais colunas](#7)
8. [Crie uma categorização](#8)
9. [Renomear colunas](#9)

# <font color = "Green"> 0. Importando os pacostes<a name = "pacotes">

In [1]:
import pandas as pd
import requests
import numpy as np

# <font color = "Green"> 1. Carregar a base <a name= "base"></a>

In [2]:
# 1) seu código aqui
sinasc = pd.read_csv('SINASC_RO_2019.csv')
print(sinasc.shape)
sinasc.drop_duplicates(inplace=True)
# Não há duplicados
print(sinasc.shape)

(27028, 69)
(27028, 69)


In [3]:
sinasc.head()

,ORIGEM,CODESTAB,CODMUNNASC,LOCNASC,IDADEMAE,ESTCIVMAE,ESCMAE,CODOCUPMAE,QTDFILVIVO,QTDFILMORT,...,KOTELCHUCK,CONTADOR,munResStatus,munResTipo,munResNome,munResUf,munResLat,munResLon,munResAlt,munResArea
0,1,2679477.0,110001,1,19,5.0,8 a 11 anos,NaN,0.0,0.0,...,5,1,ATIVO,MUNIC,Alta Floresta D'Oeste,Rondônia,-11.93554,-61.99982,338.0,7067.025
1,1,2679477.0,110001,1,29,2.0,8 a 11 anos,999992.0,1.0,0.0,...,5,2,ATIVO,MUNIC,Alta Floresta D'Oeste,Rondônia,-11.93554,-61.99982,338.0,7067.025
2,1,2679477.0,110001,1,37,9.0,8 a 11 anos,513205.0,2.0,0.0,...,5,3,ATIVO,MUNIC,Alta Floresta D'Oeste,Rondônia,-11.93554,-61.99982,338.0,7067.025
3,1,2516500.0,110001,1,30,5.0,12 anos ou mais,231205.0,0.0,0.0,...,4,4,ATIVO,MUNIC,Alto Alegre dos Parecis,Rondônia,-12.13178,-61.85308,397.0,3958.273
4,1,2516500.0,110001,1,30,2.0,8 a 11 anos,999992.0,1.0,0.0,...,5,5,ATIVO,MUNIC,Alta Floresta D'Oeste,Rondônia,-11.93554,-61.99982,338.0,7067.025


## <font color = "Green"> 2. Contar vazios por variavel <a name = "2"></a>

In [4]:
# 2) seu código aqui
pd.DataFrame(sinasc.isna().sum())

,0
ORIGEM,0
CODESTAB,115
CODMUNNASC,0
LOCNASC,0
IDADEMAE,0
...,...
munResUf,0
munResLat,1
munResLon,1
munResAlt,1


In [5]:
pd.DataFrame(sinasc.isna().sum().sort_values())

,0
ORIGEM,0
DTRECEBIM,0
DTCADASTRO,0
DIFDATA,0
PESO,0
...,...
DTULTMENST,10072
SERIESCMAE,12710
IDADEPAI,19421
CODANOMAL,26814


# <font color = "Green"> 3. Nova base e Recontagem <a name = "3"></a>

In [6]:
# 3) seu código aqui
sinasc = sinasc[['LOCNASC', 'IDADEMAE', 'ESTCIVMAE', 'ESCMAE', 'QTDFILVIVO', 
 'GESTACAO', 'GRAVIDEZ', 'CONSULTAS', 'APGAR5']]
sinasc.head()

,LOCNASC,IDADEMAE,ESTCIVMAE,ESCMAE,QTDFILVIVO,GESTACAO,GRAVIDEZ,CONSULTAS,APGAR5
0,1,19,5.0,8 a 11 anos,0.0,37 a 41 semanas,Única,4,10.0
1,1,29,2.0,8 a 11 anos,1.0,37 a 41 semanas,Única,4,9.0
2,1,37,9.0,8 a 11 anos,2.0,37 a 41 semanas,Única,4,10.0
3,1,30,5.0,12 anos ou mais,0.0,37 a 41 semanas,Única,3,10.0
4,1,30,2.0,8 a 11 anos,1.0,37 a 41 semanas,Única,4,10.0


In [7]:
pd.DataFrame(sinasc.isna().sum().sort_values())

,0
LOCNASC,0
IDADEMAE,0
CONSULTAS,0
GRAVIDEZ,79
APGAR5,103
ESCMAE,312
ESTCIVMAE,317
GESTACAO,1232
QTDFILVIVO,1573


# <font color = "Green"> 4. Remover Apgar5 não preenchido <a name = "4"></a>

In [8]:
# 4) seu código aqui
sinasc.dropna(subset=["APGAR5"])

,LOCNASC,IDADEMAE,ESTCIVMAE,ESCMAE,QTDFILVIVO,GESTACAO,GRAVIDEZ,CONSULTAS,APGAR5
0,1,19,5.0,8 a 11 anos,0.0,37 a 41 semanas,Única,4,10.0
1,1,29,2.0,8 a 11 anos,1.0,37 a 41 semanas,Única,4,9.0
2,1,37,9.0,8 a 11 anos,2.0,37 a 41 semanas,Única,4,10.0
3,1,30,5.0,12 anos ou mais,0.0,37 a 41 semanas,Única,3,10.0
4,1,30,2.0,8 a 11 anos,1.0,37 a 41 semanas,Única,4,10.0
...,...,...,...,...,...,...,...,...,...
27023,1,32,2.0,12 anos ou mais,1.0,32 a 36 semanas,Única,4,9.0
27024,1,19,1.0,8 a 11 anos,0.0,37 a 41 semanas,Única,4,9.0
27025,1,24,2.0,8 a 11 anos,0.0,37 a 41 semanas,Única,4,10.0
27026,1,21,2.0,8 a 11 anos,1.0,32 a 36 semanas,Única,4,9.0


In [9]:
pd.DataFrame(sinasc.dropna(subset=["APGAR5"]).isna().sum())

,0
LOCNASC,0
IDADEMAE,0
ESTCIVMAE,315
ESCMAE,310
QTDFILVIVO,1566
GESTACAO,1216
GRAVIDEZ,76
CONSULTAS,0
APGAR5,0


In [10]:
sinasc["APGAR5"].dropna(inplace=True)

# <font color = "Green"> 5. Estado Civil e Consulta <a name = "5"></a>

In [11]:
# 5) seu código aqui
pd.DataFrame(sinasc["ESTCIVMAE"].value_counts())

,ESTCIVMAE
2.0,9924
1.0,8952
5.0,7180
4.0,453
9.0,145
3.0,57


In [12]:
pd.DataFrame(sinasc["CONSULTAS"].value_counts())

,CONSULTAS
4,18624
3,5988
2,1379
1,986
9,51


# <font color = "Green"> **6. Substituir os valores faltantes (QTDFILVIVO)** <a name = "6"></a>

In [13]:
# 6) Seu código aqui
sinasc["QTDFILVIVO"].fillna(0,inplace=True)

In [14]:
pd.DataFrame(sinasc.isna().sum())

,0
LOCNASC,0
IDADEMAE,0
ESTCIVMAE,317
ESCMAE,312
QTDFILVIVO,0
GESTACAO,1232
GRAVIDEZ,79
CONSULTAS,0
APGAR5,103


# <font color = "Green"> 7. Substituir os valores das demais colunas <a name = "7"></a>

In [15]:
sinasc["ESTCIVMAE"].value_counts()

2.0    9924
1.0    8952
5.0    7180
4.0     453
9.0     145
3.0      57
Name: ESTCIVMAE, dtype: int64

In [16]:
# 7) seu código aqui
sinasc["ESTCIVMAE"].fillna(sinasc["ESTCIVMAE"].mean(),inplace=True) # Como é colina numerica fez sentido a media
sinasc["ESCMAE"].fillna(method="ffill",inplace=True) # Apenas um teste do metodo ffill
sinasc["GRAVIDEZ"].fillna(method="bfill",inplace=True) # Apenas um teste do metodo bfill
sinasc.dropna(inplace=True)

In [17]:
pd.DataFrame(sinasc.isna().sum())

,0
LOCNASC,0
IDADEMAE,0
ESTCIVMAE,0
ESCMAE,0
QTDFILVIVO,0
GESTACAO,0
GRAVIDEZ,0
CONSULTAS,0
APGAR5,0


In [18]:
sinasc.dtypes

LOCNASC         int64
IDADEMAE        int64
ESTCIVMAE     float64
ESCMAE         object
QTDFILVIVO    float64
GESTACAO       object
GRAVIDEZ       object
CONSULTAS       int64
APGAR5        float64
dtype: object

In [19]:
type(sinasc)

pandas.core.frame.DataFrame

# <font color = "Green"> 8. Crie uma categorização <a name = "8"></a>

1. Entre 8 e 10 está em uma faixa 'normal'.
2. Entre 6 e 7, significa que o recém-nascido passou por 'asfixia leve'.
3. Entre 4 e 5 significa 'asfixia moderada'.
4. Entre 0 e 3 significa 'asfixia severa'.

In [26]:
# 8) seu código aqui
sinasc.loc[sinasc["APGAR5"]<4,"Categoria"] = 'asfixia severa'
sinasc.loc[(sinasc["APGAR5"]>=4) & (sinasc["APGAR5"])<6,"Categoria"] = 'asfixia moderada'
sinasc.loc[(sinasc["APGAR5"]>=6) & (sinasc["APGAR5"])<=7,"Categoria"] = "asfixia leve"
sinasc.loc[sinasc["APGAR5"]>=8,"Categoria"] = "normal"


In [21]:
sinasc.head(10)

,LOCNASC,IDADEMAE,ESTCIVMAE,ESCMAE,QTDFILVIVO,GESTACAO,GRAVIDEZ,CONSULTAS,APGAR5,Categoria
0,1,19,5.0,8 a 11 anos,0.0,37 a 41 semanas,Única,4,10.0,normal
1,1,29,2.0,8 a 11 anos,1.0,37 a 41 semanas,Única,4,9.0,normal
2,1,37,9.0,8 a 11 anos,2.0,37 a 41 semanas,Única,4,10.0,normal
3,1,30,5.0,12 anos ou mais,0.0,37 a 41 semanas,Única,3,10.0,normal
4,1,30,2.0,8 a 11 anos,1.0,37 a 41 semanas,Única,4,10.0,normal
5,1,33,5.0,1 a 3 anos,2.0,37 a 41 semanas,Única,4,10.0,normal
6,1,37,1.0,8 a 11 anos,1.0,37 a 41 semanas,Única,4,10.0,normal
7,1,27,2.0,12 anos ou mais,0.0,37 a 41 semanas,Única,4,10.0,normal
8,1,20,5.0,8 a 11 anos,0.0,37 a 41 semanas,Única,3,9.0,normal
9,1,30,2.0,1 a 3 anos,1.0,37 a 41 semanas,Única,4,10.0,normal


In [27]:
sinasc["Categoria"].value_counts()

normal          25274
asfixia leve      435
Name: Categoria, dtype: int64

# <font color = "Green"> 9. Renomear colunas <a name = "9"></a>

In [24]:
# 9) seu código aqui
sinasc.columns


Index(['LOCNASC', 'IDADEMAE', 'ESTCIVMAE', 'ESCMAE', 'QTDFILVIVO', 'GESTACAO',
       'GRAVIDEZ', 'CONSULTAS', 'APGAR5', 'Categoria'],
      dtype='object')

In [25]:
sinasc.columns=['Local Nascimento', 'IDADEMAE', 'ESTCIVMAE', 'ESCMAE', 'QTDFILVIVO', 'GESTACAO',
       'GRAVIDEZ', 'CONSULTAS', 'APGAR5', 'Categoria']
sinasc

,Local Nascimento,IDADEMAE,ESTCIVMAE,ESCMAE,QTDFILVIVO,GESTACAO,GRAVIDEZ,CONSULTAS,APGAR5,Categoria
0,1,19,5.0,8 a 11 anos,0.0,37 a 41 semanas,Única,4,10.0,normal
1,1,29,2.0,8 a 11 anos,1.0,37 a 41 semanas,Única,4,9.0,normal
2,1,37,9.0,8 a 11 anos,2.0,37 a 41 semanas,Única,4,10.0,normal
3,1,30,5.0,12 anos ou mais,0.0,37 a 41 semanas,Única,3,10.0,normal
4,1,30,2.0,8 a 11 anos,1.0,37 a 41 semanas,Única,4,10.0,normal
...,...,...,...,...,...,...,...,...,...,...
27023,1,32,2.0,12 anos ou mais,1.0,32 a 36 semanas,Única,4,9.0,normal
27024,1,19,1.0,8 a 11 anos,0.0,37 a 41 semanas,Única,4,9.0,normal
27025,1,24,2.0,8 a 11 anos,0.0,37 a 41 semanas,Única,4,10.0,normal
27026,1,21,2.0,8 a 11 anos,1.0,32 a 36 semanas,Única,4,9.0,normal
